South Florida elevation map

In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import math
from config import mapbox_api_key

Set up constants

In [2]:
# Mapbox API URL
MAP_BOX_URL = 'https://api.mapbox.com/v4/mapbox.terrain-rgb/{zoom}/{x}/{y}.pngraw?access_token={token}'

#Creates a new image to paste into
overview = Image.new('RGB', (8192, 8192))
tiles = {}

# y-intercept (north extent = 0) from linear regression model - 2020 sea level
slr_const = 0.4

# NOAA VDatum API with Lat, and Lon inserted from spot off the coast of the area
vdatum_url = "https://vdatum.noaa.gov/vdatumweb/api/tidal?lon=-80.1976&lat=25.734&s_v_frame=LMSL&t_v_frame=MHHW"

#Mercator tiles in the northwest and southeast corners of desired area
city = {'nw_tiles': [2244, 3481, 13], 'se_tiles': [2276, 3513]}

Create 2 lists of tiles from west to east (x) and tiles from north to south (y)

In [3]:
xrange = list(range(city['nw_tiles'][0], city['se_tiles'][0]))
yrange = list(range(city['nw_tiles'][1], city['se_tiles'][1]))

Make API call to vdatum API and add Mean Higher High Water level to sea level constant

In [4]:
# URL contains source datum as Local Mean Sea Level (LMSL), and target datum as Mean Higher High Water (MHHW)
tide_data = requests.get(vdatum_url)
json_string = tide_data.json()

# Height level is relative to the MHHW reference point, so convert target height to float, take the absolute value and round to 2 places
flood_zone = round(abs(float(json_string['tar_height'])), 2) + slr_const
flood_zone

0.71

Define fetch tile function to make request to Mapbox API for elevation-rgb

In [5]:
def fetch_tile(x, y, zoom=city['nw_tiles'][2]):
    url = MAP_BOX_URL.format(x=x, y=y, zoom=zoom, token=mapbox_api_key)
    r = requests.get(url)
    c = r.content
    if c[0] == 123:
        return None
    return Image.open(BytesIO(c))

Nested for loops to retrieve tiles and paste them into blank image 

In [6]:
for x in xrange:
    for y in yrange:
        t = (x, y)
        if t not in tiles:
            tiles[t] = fetch_tile(x, y)
        if tiles[t] is not None:
            overview.paste(tiles[t], ((x - city['nw_tiles'][0]) * 256, (y - city['nw_tiles'][1])* 256))

Convert encoded height data from rgb to single elevation points

In [7]:
elevation_arr = np.array(overview).astype(np.float32)

r = elevation_arr[:, :, 0]
g = elevation_arr[:, :, 1]
b = elevation_arr[:, :, 2]

heights = ((r * 256 * 256 + g * 256 + b) * 0.1) - 10000
heights.shape, heights[0][0]

((8192, 8192), 2.5)

Define function to convert heights back into rgb values that will display relevant height colors

In [8]:
def height_to_rgb(x):
    if (x > 0) & (x <= slr_const):
        return np.array([255, 0, 0])
    elif (x > slr_const) & (x <= flood_zone):
        return np.array([255, 165, 0])
    elif x > flood_zone:
        return np.array([249, 228, 183])
    else:
        return np.array([102, 178, 255])

Set up blank numpy zeros array in the shape of the heights array and convert values to colors

In [9]:
h, w = heights.shape
ele_map = np.zeros((h, w, 3), np.uint8)
for x in range(w):
    for y in range(h):
        ele_map[y][x] = height_to_rgb(heights[y][x])

Display image from the elevation map numpy array

In [10]:
Image.fromarray(ele_map)

Save image as .png

In [12]:
south_florida_img = Image.fromarray(ele_map)
south_florida_img.save('south_florida_map.png')